# Using Whitebox Workflows (WbW) to process lidar data

You may download a copy of the raw [Jupyter Notebooks](https://jupyter.org/) file (`*.ipynb`) [from here](https://github.com/jblindsay/jblindsay.github.io/blob/master/WhiteboxTutorials/WbW_tutorials/WbW_lidar.ipynb).

## Setting up the WbW environment

First, let's install the WbW library using pip

In [ ]:
pip install whitebox-workflows

Or if you have it installed already but need to update to the latest version, you may do so with:

In [ ]:
pip install whitebox-workflows -U

As you're writing a WbW script, you may learn about various functions and get help with WbW idioms from the [Whitebox Workflows for Python user manual](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/preface.html).

Each WbW script must begin by importing the `WbEnvrionment` class from the `whitebox_workflows` library and setting up a `WbEnvironment` object. Also note that we are setting the `wbe.verbose` variable to `True`, which will allow the various `wbe` functions to output to the terminal. That way we can receive updates as things are happening.

In [ ]:
from whitebox_workflows import WbEnvironment, download_sample_data

wbe = WbEnvironment() # This will use the standard tier of WbW
wbe.verbose = True
print(wbe.version()) # Print the version number

Let's create a new script to download some sample data. Here we'll grab the 'Kitchener_lidar' lidar dataset. The script below will download the data for us, assign the directory to which these data are downloaded to the `WbEnvironment` working directory and lastly print this location so we can know where the data are being stored. Notice that it may take a few minutes to download the data. In the event that the download takes more than a few minutes, the connection may timeout and you will receive an error. If this should happen, you may download the dataset directly [from here](http://www.whiteboxgeo.com/sample_data/Kitchener_lidar.zip) but you will need to update the `wbe.working_directory` to your download folder.

## Working with lidar data

In [ ]:
data_dir = download_sample_data('Kitchener_lidar')
wbe.working_directory = data_dir
print(f'Data have been stored in: {wbe.working_directory}')

Let's read in the lidar (LAZ) file contained in that download directory.

In [ ]:
lidar = wbe.read_lidar('Kitchener_lidar.laz')
print(f"There are {lidar.header.number_of_points} points in the lidar dataset.")

Let's get more information about this lidar tile using the [`lidar_info`](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/tool_help.html#lidar_info) function.

In [ ]:
wbe.lidar_info(lidar, output_html_file='tile_info.html')

Now let's get some information about the distribution of points within this tile. Note that if we don't specify the `input_lidar` parameter in the function below, it'll just run the function on every lidar tile (LAS, LAZ) found in the working directory.

In [ ]:
outputs = wbe.lidar_point_stats(
    input_lidar = lidar, 
    cell_size = 1.0, 
    num_points = True, 
    num_pulses = True, 
    avg_points_per_pulse = True, 
    z_range = True, 
    intensity_range = False, 
    predominant_class = False
)

# Save the newly created outputs
wbe.write_raster(outputs[0], 'num_points.tif', compress=True)
wbe.write_raster(outputs[1], 'num_pulses.tif', compress=True)
wbe.write_raster(outputs[2], 'avg_points_per_pulse.tif', compress=True)
wbe.write_raster(outputs[3], 'z_range.tif', compress=True)

Let's measure point density of last/only return points that aren't classified noise...

In [ ]:
pt_density = wbe.lidar_point_density(
    input_lidar = lidar, 
    returns_included = "last", 
    cell_size = 1.0, 
    search_radius = 2.5, 
    excluded_classes = [7, 18] # exclude classified noise points  
)
wbe.write_raster(pt_density, 'pt_density.tif', compress=True)

Some of the following operations are contained within the Whitebox Workflows Professional (WbW-Pro) tier of the WbW library. To use these functions we need to have a valid license. Here I'm creating a new instance of a WbEnvironment that uses a floating license ID (a temporary one I'm using for this tutorial) to access the WbW-Pro functions. We **must be sure to check in this floating license after we're done using it**.

In [ ]:
wbe = WbEnvironment('wbw-tutorial') # This floating license ID will be valid for another 2 weeks.
wbe.verbose = False
wbe.working_directory = data_dir
print(wbe.working_directory)

Adding RGB values to the points based on their point returns (first, intermediate, last, only) can be useful for quality control.

In [ ]:
pt_ret_colourized = wbe.colourize_based_on_point_returns(
    input_lidar = lidar, 
    intensity_blending_amount = 50.0
)

wbe.write_lidar(pt_ret_colourized, 'pt_ret_colourized.las') # Saved as a LAS instead of LAZ so I can more easily visualize it.

print('Done!')

Once the operation above is complete, you can visualize the resulting `pt_ret_colourized.las` file in any point cloud viewing software. I recommend using [plas.io](https://plas.io) for something quick and easy. Set the colourization to RGB and turn the intensity blending down. 

Two of the most powerful functions for manipulating lidar point clouds include: 

- [filter_lidar](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/tool_help_wbwpro.html#filter_lidar)
- [modify_lidar](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/tool_help_wbwpro.html#modify_lidar)

In [ ]:
filtered_lidar = wbe.filter_lidar(
    statement = '!is_noise && is_late && class==2 && dist_to_pt(mid_x, mid_y)<250.0', 
    input_lidar = lidar
)

wbe.write_lidar(filtered_lidar, 'filtered_lidar.las')

print('Done!')

In [ ]:
modified_lidar = wbe.modify_lidar(
    statement = 'rgb = if(is_late && class==2 && dist(xy, (mid_x, mid_y))<250.0, (255,0,0), (0,255,0))', 
    input_lidar = lidar
)

wbe.write_lidar(modified_lidar, 'modified_lidar.las')

print('Done!')

As we saw earlier, this point cloud already has ground points classified. However, let's classify them using WbW-Pro's `improved_ground_point_filter` function just as a demonstration.

In [ ]:
lidar_classified = wbe.improved_ground_point_filter(
    input = lidar, 
    block_size = 1.5, 
    max_building_size = 250.0, 
    slope_threshold = 15.0, 
    elev_threshold = 0.15, 
    classify = True, 
    preserve_classes = True
)

# Let's render the point classes using RGB values for visualization
lidar_classified = wbe.colourize_based_on_class(
    input_lidar = lidar_classified, 
    intensity_blending_amount = 50.0, 
    clr_str = '1: (0, 128, 0)'
)

wbe.write_lidar(lidar_classified, 'lidar_classified.las')

print('Done!')

We can perform a more fullsome point-cloud classification too, but it's much slower...

In [ ]:
lidar_classified = wbe.classify_lidar(
    input_lidar = lidar, 
    search_radius = 1.5, 
    grd_threshold = 0.1, 
    oto_threshold = 1.0, 
    linearity_threshold = 0.5, 
    planarity_threshold = 0.85, 
    num_iter = 30, 
    facade_threshold = 0.5
)

# Let's render the point classes using RGB values for visualization
lidar_classified = wbe.colourize_based_on_class(
    input_lidar = lidar_classified, 
    intensity_blending_amount = 50.0
)

wbe.write_lidar(lidar_classified, 'lidar_full_classified.las')

print('Done!')

Let's extract [eigenvalue features](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/tool_help_wbwpro.html#lidar_eigenvalue_features) for each point in the cloud. These are a series of point metrics that can be used to describe the neighbourhood surrounding each point. Is the neighbourhood linear, planar, or a volume?

In [ ]:
# This will create a '*.eigen' file that has the same name as the input lidar file.
# These could be very useful for deep-learning based point classification applications.
# This may be a slow-running application depending on your computer power.
wbe.lidar_eigenvalue_features(
    input_lidar = lidar, 
    num_neighbours = 150, 
    search_radius = 10.0
)

# To get a sense of what these data look like, map some of the eigenvalue features
# onto the point RGB values. This is just for visualization. In reality, you'd likely
# want to read the *.eigen file into Python using NumPy. See the docs for more info.
eigen_lidar = wbe.modify_lidar(
    statement = 'rgb=(int(linearity*255), int(planarity*255), int(sphericity*255))', 
    input_lidar = lidar
)

wbe.write_lidar(eigen_lidar, 'eigen_lidar.las')

print('Done!')

Create a digital surface model from the point cloud.

In [ ]:
dsm = wbe.lidar_digital_surface_model(
    input_lidar = lidar, 
    cell_size = 1.0, 
    search_radius = 0.5
)
wbe.write_raster(dsm, 'dsm.tif', compress=True)

print('Done!')

Okay, now let's create a digital terrain model (DTM), i.e. a bare-earth DEM based on our classified ground points. Here we're using TINing, but there are other interpolators available too.

In [ ]:
dtm = wbe.lidar_tin_gridding(
    input_lidar = lidar_classified, 
    interpolation_parameter = "elevation", 
    returns_included = "all", 
    cell_size = 1.0, 
    excluded_classes = [1,7,17,18], 
)
wbe.write_raster(dtm, 'dtm.tif', compress=True)

print('Done!')

How about a DEM of difference?

In [ ]:
dod = dsm - dtm # You can use typical math ops with Whitebox rasters
wbe.write_raster(dod, 'DoD.tif', compress=True)

print('Done!')

## Working with DTM data

Let's download a different data set to work with now.

In [ ]:
wbe.working_directory = download_sample_data('mill_brook')
print(f'Data have been stored in: {wbe.working_directory}')

Create a raster DTM from the lidar point cloud.

In [ ]:
wbe.verbose = False

lidar = wbe.read_lidar('mill_brook.laz')

dtm = wbe.lidar_tin_gridding(
    input_lidar = lidar, 
    interpolation_parameter = "elevation", 
    returns_included = "all", 
    cell_size = 1.0, 
    excluded_classes = [1,7,18], 
)
wbe.write_raster(dtm, 'dtm.tif', compress=True)

print('Done!')

Let's create a hillshade image for this DTM

In [ ]:
# Let's create a hillshade image
hillshade = wbe.multidirectional_hillshade(dtm)
wbe.write_raster(hillshade, 'hillshade.tif', compress=True)

print('Done!')

Smooth the DTM. This step normally takes some experimentation to get the parameters right, which is why
why I save the raw DEM/hillshade. Comparison on the hillshade images allows me to tweak the parameters
until I find that the output DEM has the appropriate level of smoothing that I need for my application.

In [ ]:
dtm_smoothed = wbe.feature_preserving_smoothing(dtm, filter_size=11, normal_diff_threshold=25.0, iterations=5)
wbe.write_raster(dtm_smoothed, 'dtm_smoothed.tif', compress=False) 

# We'll need to save the hillshade image to compare with the hillshade from the raw DEM to
# evaluate whether the smoothing was sufficient.
hs = wbe.multidirectional_hillshade(dtm_smoothed)
wbe.write_raster(hs, 'hillshade_smoothed.tif', compress=False) 

print('Done!')

Derive a contour coverage.

In [ ]:
contours = wbe.contours_from_raster(dtm_smoothed, contour_interval=10.0)
wbe.write_vector(contours, 'contours.shp')

print('Done!')

And how about breaklines?

In [ ]:
breaklines = wbe.breakline_mapping(dtm_smoothed, threshold=3.0, min_length=3)
wbe.write_vector(breaklines, 'breaklines.shp')

print('Done!')

## Geomorphometry from DTMs

Now let's extract some common land-surface parameters (LSPs), the basic building blocks of a geomorphometric analysis.

In [ ]:
wbe.verbose = False

# Slope and aspect are two of the most common LSPs. Notice that we're combining the writing of the
# output raster and the running of the function in one line. We won't reuse the raster objects
# created by each function and are only saving them to file and so this makes sense.
wbe.write_raster(wbe.slope(dtm_smoothed, units="degrees"), 'slope.tif', compress=True)
wbe.write_raster(wbe.aspect(dtm_smoothed), 'aspect.tif', compress=True)

# Surface curvatures describe surface shape
wbe.write_raster(wbe.profile_curvature(dtm_smoothed, log_transform=True), 'prof_curv.tif', compress=True)
wbe.write_raster(wbe.tangential_curvature(dtm_smoothed, log_transform=True), 'tan_curv.tif', compress=True)
wbe.write_raster(wbe.plan_curvature(dtm_smoothed, log_transform=True), 'plan_curv.tif', compress=True)
wbe.write_raster(wbe.minimal_curvature(dtm_smoothed, log_transform=True), 'min_curv.tif', compress=True)
wbe.write_raster(wbe.maximal_curvature(dtm_smoothed, log_transform=True), 'max_curv.tif', compress=True)
wbe.write_raster(wbe.mean_curvature(dtm_smoothed, log_transform=True), 'mean_curv.tif', compress=True)
wbe.write_raster(wbe.gaussian_curvature(dtm_smoothed, log_transform=True), 'gauss_curv.tif', compress=True)
wbe.write_raster(wbe.total_curvature(dtm_smoothed, log_transform=True), 'total_curv.tif', compress=True)

print('Done!')

The following advanced curvatures are found in WbW-Pro. To run the script below, you'll **need a valid WbW-Pro license**, otherwise you will receive an error.

In [ ]:
wbe.write_raster(wbe.accumulation_curvature(dtm_smoothed, log_transform=True), 'accum_curv.tif', compress=True)
wbe.write_raster(wbe.curvedness(dtm_smoothed, log_transform=True), 'curvedness.tif', compress=True)
wbe.write_raster(wbe.difference_curvature(dtm_smoothed, log_transform=True), 'diff_curv.tif', compress=True)
wbe.write_raster(wbe.generating_function(dtm_smoothed, log_transform=True), 'generating_function.tif', compress=True)
wbe.write_raster(wbe.horizontal_excess_curvature(dtm_smoothed, log_transform=True), 'horiz_excess_curv.tif', compress=True)
wbe.write_raster(wbe.ring_curvature(dtm_smoothed, log_transform=True), 'ring_curv.tif', compress=True)
wbe.write_raster(wbe.rotor(dtm_smoothed, log_transform=True), 'rotor.tif', compress=True)
wbe.write_raster(wbe.shape_index(dtm_smoothed), 'shape_index.tif', compress=True)
wbe.write_raster(wbe.unsphericity(dtm_smoothed, log_transform=True), 'unsphericity.tif', compress=True)
wbe.write_raster(wbe.vertical_excess_curvature(dtm_smoothed, log_transform=True), 'vertical_excess_curv.tif', compress=True)

print('Done!')

Measures of local topographic position (LTP).

In [ ]:
wbe.write_raster(
    wbe.elevation_percentile(
        dtm_smoothed, 
        filter_size_x = 51, 
        filter_size_y = 51, 
        sig_digits = 3
    ), 
    'elev_percentile.tif', 
    compress=True
)

wbe.write_raster(
    wbe.difference_from_mean_elevation(
        dtm_smoothed, 
        filter_size_x = 51, 
        filter_size_y = 51
    ), 
    'dev.tif', 
    compress=True
)

print('Done!')

The following LSPs can be used to characterize surface roughness and topographic complexity.

In [ ]:
wbe.write_raster(wbe.circular_variance_of_aspect(dtm_smoothed, filter_size = 21), 'circular_variance_of_aspect.tif', compress=True)
wbe.write_raster(wbe.edge_density(dtm_smoothed, filter_size=21, normal_diff_threshold=5.0), 'edge_density.tif', compress=True)
wbe.write_raster(wbe.spherical_std_dev_of_normals(dtm_smoothed, filter_size = 21), 'spherical_sd_norms.tif', compress=True)
wbe.write_raster(wbe.standard_deviation_of_slope(dtm_smoothed, filter_size = 21), 'stdev_slope.tif', compress=True)
wbe.write_raster(wbe.surface_area_ratio(dtm_smoothed), 'surface_area_ratio.tif', compress=True)
wbe.write_raster(wbe.ruggedness_index(dtm_smoothed), 'ruggedness_index.tif', compress=True)

print('Done!')

## Spatial hydrology

Now let's do a bit of hydrological processing of the data, including extracting a stream network.

In [ ]:
import math # We'll use the log function below

# Remove the depressions, first by breaching the depressions using a max dist so that it doesn't
# carve excessively long trenches for very deep pits, and then filling the remaining depressions
dem_no_deps = wbe.breach_depressions_least_cost(dtm_smoothed, flat_increment=0.001, max_dist=100) # Change the max dist parameter as appropriate for your DEM
dem_no_deps = wbe.fill_depressions(dem_no_deps, flat_increment=0.001)

# Perform a flow-accumulation operation. Here I'm using the Qin (2007) multiple flow direction algorithm
# but there are many other options available, including D-infinity.
#
# Stream channels are usually identified as areas of relatively high flow accumulation and are mapped by thresholding
# flow accumulation values. Let's choose a threshold value.
channel_threshold = 25000.0
flow_accum = wbe.qin_flow_accumulation(dem_no_deps, out_type='cells', convergence_threshold=channel_threshold, log_transform=True)
wbe.write_raster(flow_accum, 'qin_flow_accum.tif')

# Map the streams by thresholding the flow accum raster, using the same convergence threshold used above. This way
# we can be assured that the streams are single-cell wide D8 representation, which is needed for any stream
# network analysis operations.
streams = flow_accum > math.log(channel_threshold)

print('Done!')

Decreasing the value of `channel_threshold` will result in a more extensive network of stream channels and increasing it will result in a less extensive network. The channel threshold of 25000 (in grid cells) has been selected simply by examining the values of flow accumulation within the `qin_flow_accum.tif` file near the headwaters of the visible stream channels in the hillshade image. There will, of course, be variation in this value and it may require some refining to get a reasonable value that performs well throughout. In fact, geomorphologists often use more sophisticated methods, usually involving slope and sometimes other factors, to select a channelization threshold. Experiment with the value of `channel_threshold` to see how the stream network is impacted by this value.

Now let's map the areas draining to an outlet point and to various parts of the stream network...

In [ ]:
# Let's extract the watershed for a specific outlet point
outlet = wbe.read_vector('outlet.shp') # This is a vector point that was included when we downloaded the `mill_brook` dataset.

# Make sure that the outlet is positioned along the stream
outlet = wbe.jenson_snap_pour_points(outlet, streams, 5.0)

# We need a d8-pointer raster to be able to route flow through the network
d8_pntr = wbe.d8_pointer(dem_no_deps)

# Extract the outlet's watershed
watershed = wbe.watershed(d8_pointer=d8_pntr, pour_points=outlet)

# Vectorize the watershed polygon for visualization
watershed_vec = wbe.raster_to_vector_polygons(watershed)
# Smooth the watershed map for visualization
watershed_vec = wbe.smooth_vectors(watershed_vec, filter_size=5) 
wbe.write_vector(watershed_vec, 'watershed.shp')

# Now, we only want the streams inside the watershed
streams = streams * watershed # Notice that we can treat the rasters like any other Python variable in a math equation.

# Perform a stream network analysis on the stream vector
streams_vec = wbe.raster_streams_to_vector(streams, d8_pntr)
streams_vec, tmp1, tmp2, tmp3 = wbe.vector_stream_network_analysis(streams_vec, dem_no_deps) # We only want the streams output
wbe.write_vector(streams_vec, 'streams.shp')

# Extract all of the watersheds, draining to each outlet on the edge of the DEM using the 'basins' function.
basins = wbe.basins(d8_pntr)
wbe.write_raster(basins, 'basins.tif')

# How about extracting subcatchments, i.e. the areas draining directly to each link in the stream network?
subcatchments = wbe.subbasins(d8_pntr, streams)
wbe.write_raster(subcatchments, 'subcatchments.tif')

# Or perhaps map Strahler basins, i.e. the areas draining to Strahler order 1, 2, 3, etc. streams...
strahler_basins = wbe.strahler_order_basins(d8_pointer=d8_pntr, streams=streams)
wbe.write_raster(strahler_basins, 'strahler_basins.tif')

print('Done!')

Let's calculate depth-to-water index, useful for flood mapping applications.

In [ ]:
depth_to_water = wbe.depth_to_water(
    dem = dem_no_deps, 
    streams = streams_vec
)
wbe.write_raster(depth_to_water, 'depth_to_water.tif')

print('Done!')

## Wrapping things up

Don't forget to check in your WbW-Pro floating license or it won't be available later.

In [ ]:
wbe.check_in_license('wbw-tutorial')